In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Klue 데이터 원본 가져오기

In [41]:
from pathlib import Path
import json

def read_klue(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    guids =[]
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                contexts.append(context)
                questions.append(question)
                a_list=[]
                if qa['answers']:
                  for a in qa['answers']:
                    a_list.append(a["text"])
                else:
                  for a in qa["plausible_answers"]:
                     a_list.append(a["text"])

                answers.append(a_list)
                guids.append(qa["guid"])

    return guids ,contexts, questions, answers


In [42]:
import pandas as pd

klue_train_file_path =  '/content/drive/MyDrive/Goorm_Deep_Learning/Projects/project2/Data/kluemrc/klue-mrc-v1.1_train.json'

guids, contexts, questions, answers = read_klue(klue_train_file_path )
klue_train = pd.DataFrame(list(zip(guids,answers, contexts, questions)),
               columns =['guid','Answer', 'Context','Question'])


In [43]:
import pandas as pd

klue_dev_file_path =  '/content/drive/MyDrive/Goorm_Deep_Learning/Projects/project2/Data/kluemrc/klue-mrc-v1.1_dev.json'

guids, contexts, questions, answers = read_klue(klue_dev_file_path )
klue_dev = pd.DataFrame(list(zip(guids,answers, contexts, questions)),
               columns =['guid','Answer', 'Context','Question'])


In [44]:
klue_train

,guid,Answer,Context,Question
0,klue-mrc-v1_train_12759,"[한 달가량, 한 달]",올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...,북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
1,klue-mrc-v1_train_06118,"[원장 이인숙, 이인숙, 이인숙]",부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...,성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는?
2,klue-mrc-v1_train_14934,[삼보테크놀로지],부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...,지능형 생산자동화 기반기술을 개발중인 스타트업은?
3,klue-mrc-v1_train_02900,[레이다],"미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에...",로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?
4,klue-mrc-v1_train_02773,[와쿠이 히데아키],시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...,개막전에서 3안타 2실점을 기록해서 패한 선수는?
...,...,...,...,...
17549,klue-mrc-v1_train_08088,"[1.02%, 1.02]",김주하 농협은행장(사진)이 여신담당 인력을 2800여명에서 2017년까지 5000명...,김주하 농협은행장 취임 3년전의 연체율은?
17550,klue-mrc-v1_train_09135,[롯데백화점 잠실점],홈 뷰티 디바이스 브랜드 ‘엘리닉(L.linic)’이 고객 감사의 달을 맞아 전국 ...,팝업스토어 앵콜 행사를 가장 먼저 시작하는 곳은 어디인가?
17551,klue-mrc-v1_train_14709,"[‘혹성탈출: 반격의 서막’, 혹성탈출: 반격의 서막]",유인원 무리의 리더 시저는 인간 건축가 말콤(제이슨 클락)에게 작별 인사를 한다. ...,혹성탈출의 두 번째 프리퀄의 제목은?
17552,klue-mrc-v1_train_07955,[8시 10분],ASUS(에이수스) 그래픽카드 공식수입사 인텍앤컴퍼니(대표 서정욱)는 10월 16일...,인택엔컴퍼니가 실시하는 추첨판매 신청 마감시간은?


In [45]:
klue = pd.concat([klue_train,klue_dev])

## 구름 Test dataset에서 ID 와 question 추출

In [46]:
from pathlib import Path
import json

def read_klue_test(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    guids = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                contexts.append(context)
                questions.append(question)
                answers.append(None)
                guids.append(qa["guid"])

    return guids,contexts, questions, answers

In [47]:
import pandas as pd

file_path =  '/content/drive/MyDrive/Goorm_Deep_Learning/Projects/project2/Data/test.json'

guids, contexts, questions, answers = read_klue_test(file_path)
test_df = pd.DataFrame(list(zip(guids,questions)),
               columns =['Id','Question'])
test_df

,Id,Question
0,d14cb73158624cf094c546d856fd3c80,말라카이트에서 나온 색깔을 사용한 에디션은?
1,906631384e91493ebe1c7f34aea6f241,사고 비행기의 목적지는?
2,35e61dcb479643448a2cb7d326ae50a6,2014년 일하고 싶은 50대 회사 중에서 5위로 선정된 기업은?
3,075e761b370040cb9041eecd39afc27c,포브스의 2014년 일하고 싶은 50대 회사 조사에서 5위를 한 기업은?
4,e67ed38f3dd944be94d5b4c53731f334,호세 카레라스가 재단의 도움을 받아 병을 치료한 병원의 소재지는?
...,...,...
4003,05fcb8054dc44dab8683579c2cf5e465,도쿄지점의 현재 개인 신용대출 한도는?
4004,cc7f826b66724ce9b39e3a974ca15661,오목이 수지에게 청탁한 남편의 일자리 장소는?
4005,3282034aa41e4fab980851ffd4a868dd,컨테이너선 평균 운임이 15%정도 낮아진 노선은?
4006,0a73550b36df4baf82ac2f98619d22e7,강남지역에 사는 학생들은 며칠 동안 학교를 안가나?


## Klue 원본 data와 구름 Test data의 question을 비교하여 정답지 만들기  

In [48]:
test_answer = pd.merge(klue,test_df)
test_answer

,guid,Answer,Context,Question,Id
0,klue-mrc-v1_train_05598,[터키 이스탄불],일본 도쿄가 원전 오염수 유출이라는 대형 악재에도 불구하고 2020년 하계올림픽 개...,2020년 하계올림픽 개최지를 결정하는 투표에서 도쿄에게 밀린 곳은?,6727d61923604c60b41d864c00c1a89a
1,klue-mrc-v1_dev_05696,[블루스퀘어],지난해 한국어로 공연된 뮤지컬 중 최고 흥행작이자 화제작은 ‘엘리자벳’이었다. 오스...,2020년 하계올림픽 개최지를 결정하는 투표에서 도쿄에게 밀린 곳은?,6727d61923604c60b41d864c00c1a89a
2,klue-mrc-v1_dev_01891,"[뉴 740Li 25주년 에디션, 뉴 740Li 25주년]",BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에...,말라카이트에서 나온 색깔을 사용한 에디션은?,d14cb73158624cf094c546d856fd3c80
3,klue-mrc-v1_dev_01842,[독일 뒤셀도르프],프랑스 남부 알프스에 떨어져 150명의 사망자를 낸 저먼윙스 여객기는 부조종사가 의...,사고 비행기의 목적지는?,906631384e91493ebe1c7f34aea6f241
4,klue-mrc-v1_dev_02673,[페이스북],가장 일하고 싶은 회사로 미국 컨설팅 업체인 베인&컴퍼니가 선정됐다. 트위터와 링크...,2014년 일하고 싶은 50대 회사 중에서 5위로 선정된 기업은?,35e61dcb479643448a2cb7d326ae50a6
...,...,...,...,...,...
4004,klue-mrc-v1_dev_03796,[200만엔],일본 도쿄지점 130억원대 부당대출 혐의로 금융감독원의 검사를 받고 있는 기업은행이...,도쿄지점의 현재 개인 신용대출 한도는?,05fcb8054dc44dab8683579c2cf5e465
4005,klue-mrc-v1_dev_00770,"[중동 건설 현장, 중동]",한국전쟁 당시 언니인 수지는 여동생 오목이를 고의로 방치했다. 수지는 신경쓰지 않고...,오목이 수지에게 청탁한 남편의 일자리 장소는?,cc7f826b66724ce9b39e3a974ca15661
4006,klue-mrc-v1_dev_04893,"[아시아~유럽 노선, 아시아~유럽]",세계 컨테이너선 운임지수가 사상 최저 수준으로 떨어지면서 해운업계에 비상이 걸렸다....,컨테이너선 평균 운임이 15%정도 낮아진 노선은?,3282034aa41e4fab980851ffd4a868dd
4007,klue-mrc-v1_dev_04183,[사흘],"서울교육청이 8일부터 10일까지 강남구, 서초구 지역에 있는 유치원, 초등학교의 전...",강남지역에 사는 학생들은 며칠 동안 학교를 안가나?,0a73550b36df4baf82ac2f98619d22e7


## 편집 거리 계산 함수

In [49]:
#https://lovit.github.io/nlp/2018/08/28/levenshtein_hangle/
def levenshtein(s1, s2, debug=False):
    if len(s1) < len(s2):
        return levenshtein(s2, s1, debug)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))

        if debug:
            print(current_row[1:])

        previous_row = current_row

    return previous_row[-1]

## 모두 빈칸인 경우로 측정

In [50]:
test_answer_temp = test_answer.copy()
test_answer_temp["Predicted"]=[""]*len(test_answer_temp)

In [51]:
score_list = []
for index, row in test_answer_temp.iterrows():
  prediction = row["Predicted"]
  answer = row["Answer"][0]
  score = levenshtein(answer, prediction)
  score_list.append(score)



In [52]:
test_answer_temp["levenshtein"] = score_list
test_answer_temp

,guid,Answer,Context,Question,Id,Predicted,levenshtein
0,klue-mrc-v1_train_05598,[터키 이스탄불],일본 도쿄가 원전 오염수 유출이라는 대형 악재에도 불구하고 2020년 하계올림픽 개...,2020년 하계올림픽 개최지를 결정하는 투표에서 도쿄에게 밀린 곳은?,6727d61923604c60b41d864c00c1a89a,,7
1,klue-mrc-v1_dev_05696,[블루스퀘어],지난해 한국어로 공연된 뮤지컬 중 최고 흥행작이자 화제작은 ‘엘리자벳’이었다. 오스...,2020년 하계올림픽 개최지를 결정하는 투표에서 도쿄에게 밀린 곳은?,6727d61923604c60b41d864c00c1a89a,,5
2,klue-mrc-v1_dev_01891,"[뉴 740Li 25주년 에디션, 뉴 740Li 25주년]",BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에...,말라카이트에서 나온 색깔을 사용한 에디션은?,d14cb73158624cf094c546d856fd3c80,,16
3,klue-mrc-v1_dev_01842,[독일 뒤셀도르프],프랑스 남부 알프스에 떨어져 150명의 사망자를 낸 저먼윙스 여객기는 부조종사가 의...,사고 비행기의 목적지는?,906631384e91493ebe1c7f34aea6f241,,8
4,klue-mrc-v1_dev_02673,[페이스북],가장 일하고 싶은 회사로 미국 컨설팅 업체인 베인&컴퍼니가 선정됐다. 트위터와 링크...,2014년 일하고 싶은 50대 회사 중에서 5위로 선정된 기업은?,35e61dcb479643448a2cb7d326ae50a6,,4
...,...,...,...,...,...,...,...
4004,klue-mrc-v1_dev_03796,[200만엔],일본 도쿄지점 130억원대 부당대출 혐의로 금융감독원의 검사를 받고 있는 기업은행이...,도쿄지점의 현재 개인 신용대출 한도는?,05fcb8054dc44dab8683579c2cf5e465,,5
4005,klue-mrc-v1_dev_00770,"[중동 건설 현장, 중동]",한국전쟁 당시 언니인 수지는 여동생 오목이를 고의로 방치했다. 수지는 신경쓰지 않고...,오목이 수지에게 청탁한 남편의 일자리 장소는?,cc7f826b66724ce9b39e3a974ca15661,,8
4006,klue-mrc-v1_dev_04893,"[아시아~유럽 노선, 아시아~유럽]",세계 컨테이너선 운임지수가 사상 최저 수준으로 떨어지면서 해운업계에 비상이 걸렸다....,컨테이너선 평균 운임이 15%정도 낮아진 노선은?,3282034aa41e4fab980851ffd4a868dd,,9
4007,klue-mrc-v1_dev_04183,[사흘],"서울교육청이 8일부터 10일까지 강남구, 서초구 지역에 있는 유치원, 초등학교의 전...",강남지역에 사는 학생들은 며칠 동안 학교를 안가나?,0a73550b36df4baf82ac2f98619d22e7,,2


In [53]:
sum(score_list)/4009

5.937390870541282

## 예측한 결과로 성능 측정하기

In [54]:
#colnames = ["id","prediction"]

## 모델의 결과가 담긴 파일
#result = pd.read_csv('/content/drive/MyDrive/Goorm_Deep_Learning/Projects/project2/Result/KoELECTRA_result.csv',on_bad_lines='skip',engine="python",names = colnames)
PATH=  '/content/drive/MyDrive/Goorm_Deep_Learning/Projects/project2/Result/koelectra-base-v3-discriminator_result'

result = pd.read_csv(PATH+'.csv',on_bad_lines='skip')

In [55]:
result

,Id,Context,Question,Predicted,Score
0,d14cb73158624cf094c546d856fd3c80,BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에...,말라카이트에서 나온 색깔을 사용한 에디션은?,뉴 740Li 25주년 에디션,0.141827
1,906631384e91493ebe1c7f34aea6f241,프랑스 남부 알프스에 떨어져 150명의 사망자를 낸 저먼윙스 여객기는 부조종사가 의...,사고 비행기의 목적지는?,독일 뒤셀도르프로,0.595488
2,35e61dcb479643448a2cb7d326ae50a6,가장 일하고 싶은 회사로 미국 컨설팅 업체인 베인&컴퍼니가 선정됐다. 트위터와 링크...,2014년 일하고 싶은 50대 회사 중에서 5위로 선정된 기업은?,꾸준히,0.949676
3,075e761b370040cb9041eecd39afc27c,가장 일하고 싶은 회사로 미국 컨설팅 업체인 베인&컴퍼니가 선정됐다. 트위터와 링크...,포브스의 2014년 일하고 싶은 50대 회사 조사에서 5위를 한 기업은?,꾸준히,0.945235
4,e67ed38f3dd944be94d5b4c53731f334,성악가로서 한창 전성기를 구가하던 1987년 호세 카레라스는 청천벽력같은 소식을 듣...,호세 카레라스가 재단의 도움을 받아 병을 치료한 병원의 소재지는?,마드리드에,0.940322
...,...,...,...,...,...
4003,05fcb8054dc44dab8683579c2cf5e465,일본 도쿄지점 130억원대 부당대출 혐의로 금융감독원의 검사를 받고 있는 기업은행이...,도쿄지점의 현재 개인 신용대출 한도는?,2만달러로,0.289632
4004,cc7f826b66724ce9b39e3a974ca15661,한국전쟁 당시 언니인 수지는 여동생 오목이를 고의로 방치했다. 수지는 신경쓰지 않고...,오목이 수지에게 청탁한 남편의 일자리 장소는?,중동 건설 현장으로,0.627388
4005,3282034aa41e4fab980851ffd4a868dd,세계 컨테이너선 운임지수가 사상 최저 수준으로 떨어지면서 해운업계에 비상이 걸렸다....,컨테이너선 평균 운임이 15%정도 낮아진 노선은?,아시아~유럽 노선의,0.454696
4006,0a73550b36df4baf82ac2f98619d22e7,"서울교육청이 8일부터 10일까지 강남구, 서초구 지역에 있는 유치원, 초등학교의 전...",강남지역에 사는 학생들은 며칠 동안 학교를 안가나?,12일까지,0.303501


In [56]:
#result = result[["Question","Predicted"]]

In [57]:
result.isna().sum()

Id           0
Context      0
Question     0
Predicted    0
Score        0
dtype: int64

In [58]:
# 빈칸 삭제
#result = result.dropna()
result

,Id,Context,Question,Predicted,Score
0,d14cb73158624cf094c546d856fd3c80,BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에...,말라카이트에서 나온 색깔을 사용한 에디션은?,뉴 740Li 25주년 에디션,0.141827
1,906631384e91493ebe1c7f34aea6f241,프랑스 남부 알프스에 떨어져 150명의 사망자를 낸 저먼윙스 여객기는 부조종사가 의...,사고 비행기의 목적지는?,독일 뒤셀도르프로,0.595488
2,35e61dcb479643448a2cb7d326ae50a6,가장 일하고 싶은 회사로 미국 컨설팅 업체인 베인&컴퍼니가 선정됐다. 트위터와 링크...,2014년 일하고 싶은 50대 회사 중에서 5위로 선정된 기업은?,꾸준히,0.949676
3,075e761b370040cb9041eecd39afc27c,가장 일하고 싶은 회사로 미국 컨설팅 업체인 베인&컴퍼니가 선정됐다. 트위터와 링크...,포브스의 2014년 일하고 싶은 50대 회사 조사에서 5위를 한 기업은?,꾸준히,0.945235
4,e67ed38f3dd944be94d5b4c53731f334,성악가로서 한창 전성기를 구가하던 1987년 호세 카레라스는 청천벽력같은 소식을 듣...,호세 카레라스가 재단의 도움을 받아 병을 치료한 병원의 소재지는?,마드리드에,0.940322
...,...,...,...,...,...
4003,05fcb8054dc44dab8683579c2cf5e465,일본 도쿄지점 130억원대 부당대출 혐의로 금융감독원의 검사를 받고 있는 기업은행이...,도쿄지점의 현재 개인 신용대출 한도는?,2만달러로,0.289632
4004,cc7f826b66724ce9b39e3a974ca15661,한국전쟁 당시 언니인 수지는 여동생 오목이를 고의로 방치했다. 수지는 신경쓰지 않고...,오목이 수지에게 청탁한 남편의 일자리 장소는?,중동 건설 현장으로,0.627388
4005,3282034aa41e4fab980851ffd4a868dd,세계 컨테이너선 운임지수가 사상 최저 수준으로 떨어지면서 해운업계에 비상이 걸렸다....,컨테이너선 평균 운임이 15%정도 낮아진 노선은?,아시아~유럽 노선의,0.454696
4006,0a73550b36df4baf82ac2f98619d22e7,"서울교육청이 8일부터 10일까지 강남구, 서초구 지역에 있는 유치원, 초등학교의 전...",강남지역에 사는 학생들은 며칠 동안 학교를 안가나?,12일까지,0.303501


In [59]:
test_answer_prediction = pd.merge(test_answer,result)
test_answer_prediction

,guid,Answer,Context,Question,Id,Predicted,Score
0,klue-mrc-v1_dev_05696,[블루스퀘어],지난해 한국어로 공연된 뮤지컬 중 최고 흥행작이자 화제작은 ‘엘리자벳’이었다. 오스...,2020년 하계올림픽 개최지를 결정하는 투표에서 도쿄에게 밀린 곳은?,6727d61923604c60b41d864c00c1a89a,캐스팅에,0.001435
1,klue-mrc-v1_dev_01891,"[뉴 740Li 25주년 에디션, 뉴 740Li 25주년]",BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에...,말라카이트에서 나온 색깔을 사용한 에디션은?,d14cb73158624cf094c546d856fd3c80,뉴 740Li 25주년 에디션,0.141827
2,klue-mrc-v1_dev_01842,[독일 뒤셀도르프],프랑스 남부 알프스에 떨어져 150명의 사망자를 낸 저먼윙스 여객기는 부조종사가 의...,사고 비행기의 목적지는?,906631384e91493ebe1c7f34aea6f241,독일 뒤셀도르프로,0.595488
3,klue-mrc-v1_dev_02673,[페이스북],가장 일하고 싶은 회사로 미국 컨설팅 업체인 베인&컴퍼니가 선정됐다. 트위터와 링크...,2014년 일하고 싶은 50대 회사 중에서 5위로 선정된 기업은?,35e61dcb479643448a2cb7d326ae50a6,꾸준히,0.949676
4,klue-mrc-v1_dev_05664,[페이스북],가장 일하고 싶은 회사로 미국 컨설팅 업체인 베인&컴퍼니가 선정됐다. 트위터와 링크...,포브스의 2014년 일하고 싶은 50대 회사 조사에서 5위를 한 기업은?,075e761b370040cb9041eecd39afc27c,꾸준히,0.945235
...,...,...,...,...,...,...,...
4003,klue-mrc-v1_dev_03796,[200만엔],일본 도쿄지점 130억원대 부당대출 혐의로 금융감독원의 검사를 받고 있는 기업은행이...,도쿄지점의 현재 개인 신용대출 한도는?,05fcb8054dc44dab8683579c2cf5e465,2만달러로,0.289632
4004,klue-mrc-v1_dev_00770,"[중동 건설 현장, 중동]",한국전쟁 당시 언니인 수지는 여동생 오목이를 고의로 방치했다. 수지는 신경쓰지 않고...,오목이 수지에게 청탁한 남편의 일자리 장소는?,cc7f826b66724ce9b39e3a974ca15661,중동 건설 현장으로,0.627388
4005,klue-mrc-v1_dev_04893,"[아시아~유럽 노선, 아시아~유럽]",세계 컨테이너선 운임지수가 사상 최저 수준으로 떨어지면서 해운업계에 비상이 걸렸다....,컨테이너선 평균 운임이 15%정도 낮아진 노선은?,3282034aa41e4fab980851ffd4a868dd,아시아~유럽 노선의,0.454696
4006,klue-mrc-v1_dev_04183,[사흘],"서울교육청이 8일부터 10일까지 강남구, 서초구 지역에 있는 유치원, 초등학교의 전...",강남지역에 사는 학생들은 며칠 동안 학교를 안가나?,0a73550b36df4baf82ac2f98619d22e7,12일까지,0.303501


In [70]:
# 정답과 예측값의 편집거리를 계산하여 리스트에 저장
score_list = []
for index, row in test_answer_prediction.iterrows():
  prediction = row["Predicted"]
  answer = row["Answer"][0]
  score = levenshtein(answer, prediction)
  score_list.append(score)

In [61]:
# 평균 편집거리
sum(score_list)/len(test_answer_prediction)

5.938872255489022

In [63]:
test_answer_prediction["levenshtein"] = score_list

In [ ]:
#체점 결과 저장하기
#test_answer_prediction.to_csv(PATH+"_check.csv",index = False)

In [64]:
test_answer_prediction

,guid,Answer,Context,Question,Id,Predicted,Score
0,klue-mrc-v1_dev_05696,[블루스퀘어],지난해 한국어로 공연된 뮤지컬 중 최고 흥행작이자 화제작은 ‘엘리자벳’이었다. 오스...,2020년 하계올림픽 개최지를 결정하는 투표에서 도쿄에게 밀린 곳은?,6727d61923604c60b41d864c00c1a89a,캐스팅에,0.001435
1,klue-mrc-v1_dev_01891,"[뉴 740Li 25주년 에디션, 뉴 740Li 25주년]",BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에...,말라카이트에서 나온 색깔을 사용한 에디션은?,d14cb73158624cf094c546d856fd3c80,뉴 740Li 25주년 에디션,0.141827
2,klue-mrc-v1_dev_01842,[독일 뒤셀도르프],프랑스 남부 알프스에 떨어져 150명의 사망자를 낸 저먼윙스 여객기는 부조종사가 의...,사고 비행기의 목적지는?,906631384e91493ebe1c7f34aea6f241,독일 뒤셀도르프로,0.595488
3,klue-mrc-v1_dev_02673,[페이스북],가장 일하고 싶은 회사로 미국 컨설팅 업체인 베인&컴퍼니가 선정됐다. 트위터와 링크...,2014년 일하고 싶은 50대 회사 중에서 5위로 선정된 기업은?,35e61dcb479643448a2cb7d326ae50a6,꾸준히,0.949676
4,klue-mrc-v1_dev_05664,[페이스북],가장 일하고 싶은 회사로 미국 컨설팅 업체인 베인&컴퍼니가 선정됐다. 트위터와 링크...,포브스의 2014년 일하고 싶은 50대 회사 조사에서 5위를 한 기업은?,075e761b370040cb9041eecd39afc27c,꾸준히,0.945235
...,...,...,...,...,...,...,...
4003,klue-mrc-v1_dev_03796,[200만엔],일본 도쿄지점 130억원대 부당대출 혐의로 금융감독원의 검사를 받고 있는 기업은행이...,도쿄지점의 현재 개인 신용대출 한도는?,05fcb8054dc44dab8683579c2cf5e465,2만달러로,0.289632
4004,klue-mrc-v1_dev_00770,"[중동 건설 현장, 중동]",한국전쟁 당시 언니인 수지는 여동생 오목이를 고의로 방치했다. 수지는 신경쓰지 않고...,오목이 수지에게 청탁한 남편의 일자리 장소는?,cc7f826b66724ce9b39e3a974ca15661,중동 건설 현장으로,0.627388
4005,klue-mrc-v1_dev_04893,"[아시아~유럽 노선, 아시아~유럽]",세계 컨테이너선 운임지수가 사상 최저 수준으로 떨어지면서 해운업계에 비상이 걸렸다....,컨테이너선 평균 운임이 15%정도 낮아진 노선은?,3282034aa41e4fab980851ffd4a868dd,아시아~유럽 노선의,0.454696
4006,klue-mrc-v1_dev_04183,[사흘],"서울교육청이 8일부터 10일까지 강남구, 서초구 지역에 있는 유치원, 초등학교의 전...",강남지역에 사는 학생들은 며칠 동안 학교를 안가나?,0a73550b36df4baf82ac2f98619d22e7,12일까지,0.303501


In [65]:
#test_answer_prediction=test_answer_prediction[["Id","Predicted"]]

In [66]:
#test_answer_prediction.to_csv(PATH+"_submission_check(4.29)2.csv",index=False)

## 후처리하기

In [67]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 5.0 MB/s 
     |████████████████████████████████| 448 kB 49.8 MB/s 


In [68]:
#!pip install konlpy
from konlpy.tag import Okt
df = test_answer_prediction.copy()

long_cnt = 0
low_score = 0
remove_josa = 0
for index, row in df.iterrows():
  #예측이 20문자가 넘는다면 앞의 10문자만 사용
  if len(row["Predicted"]) > 20:
    df.at[index,'Predicted'] = row["Predicted"][:10]
    long_cnt +=1
    continue
 # score가 0.05점이 넘지 않는다면 빈칸으로 대체
  if row["Score"] < 0.05:
    df.at[index,'Predicted'] = ""
    low_score +=1
    continue
  #예측의 마지막이 조사라면 조사를 제거
  tag = Okt().pos(row["Predicted"])
  if tag[-1][1] == 'Josa':
    target = tag[-1][0]
    clean = row["Predicted"].rstrip(target)
    df.at[index,'Predicted'] = clean
    remove_josa +=1

    

print("길어서 제거된 답 ",long_cnt)
print("score가 낮아서 제거된 답", low_score)
print("제거된 조사 수", remove_josa)  
  


길어서 제거된 답  46
score가 낮아서 제거된 답 40
제거된 조사 수 1513


In [69]:
df

,guid,Answer,Context,Question,Id,Predicted,Score
0,klue-mrc-v1_dev_05696,[블루스퀘어],지난해 한국어로 공연된 뮤지컬 중 최고 흥행작이자 화제작은 ‘엘리자벳’이었다. 오스...,2020년 하계올림픽 개최지를 결정하는 투표에서 도쿄에게 밀린 곳은?,6727d61923604c60b41d864c00c1a89a,,0.001435
1,klue-mrc-v1_dev_01891,"[뉴 740Li 25주년 에디션, 뉴 740Li 25주년]",BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에...,말라카이트에서 나온 색깔을 사용한 에디션은?,d14cb73158624cf094c546d856fd3c80,뉴 740Li 25주년 에디션,0.141827
2,klue-mrc-v1_dev_01842,[독일 뒤셀도르프],프랑스 남부 알프스에 떨어져 150명의 사망자를 낸 저먼윙스 여객기는 부조종사가 의...,사고 비행기의 목적지는?,906631384e91493ebe1c7f34aea6f241,독일 뒤셀도르프,0.595488
3,klue-mrc-v1_dev_02673,[페이스북],가장 일하고 싶은 회사로 미국 컨설팅 업체인 베인&컴퍼니가 선정됐다. 트위터와 링크...,2014년 일하고 싶은 50대 회사 중에서 5위로 선정된 기업은?,35e61dcb479643448a2cb7d326ae50a6,꾸준히,0.949676
4,klue-mrc-v1_dev_05664,[페이스북],가장 일하고 싶은 회사로 미국 컨설팅 업체인 베인&컴퍼니가 선정됐다. 트위터와 링크...,포브스의 2014년 일하고 싶은 50대 회사 조사에서 5위를 한 기업은?,075e761b370040cb9041eecd39afc27c,꾸준히,0.945235
...,...,...,...,...,...,...,...
4003,klue-mrc-v1_dev_03796,[200만엔],일본 도쿄지점 130억원대 부당대출 혐의로 금융감독원의 검사를 받고 있는 기업은행이...,도쿄지점의 현재 개인 신용대출 한도는?,05fcb8054dc44dab8683579c2cf5e465,2만달러로,0.289632
4004,klue-mrc-v1_dev_00770,"[중동 건설 현장, 중동]",한국전쟁 당시 언니인 수지는 여동생 오목이를 고의로 방치했다. 수지는 신경쓰지 않고...,오목이 수지에게 청탁한 남편의 일자리 장소는?,cc7f826b66724ce9b39e3a974ca15661,중동 건설 현장,0.627388
4005,klue-mrc-v1_dev_04893,"[아시아~유럽 노선, 아시아~유럽]",세계 컨테이너선 운임지수가 사상 최저 수준으로 떨어지면서 해운업계에 비상이 걸렸다....,컨테이너선 평균 운임이 15%정도 낮아진 노선은?,3282034aa41e4fab980851ffd4a868dd,아시아~유럽 노선,0.454696
4006,klue-mrc-v1_dev_04183,[사흘],"서울교육청이 8일부터 10일까지 강남구, 서초구 지역에 있는 유치원, 초등학교의 전...",강남지역에 사는 학생들은 며칠 동안 학교를 안가나?,0a73550b36df4baf82ac2f98619d22e7,12일까지,0.303501


In [71]:
# 정답과 예측값의 편집거리를 계산하여 리스트에 저장
score_list = []
for index, row in df.iterrows():
  prediction = row["Predicted"]
  answer = row["Answer"][0]
  score = levenshtein(answer, prediction)
  score_list.append(score)

# 평균 편집거리
sum(score_list)/len(test_answer_prediction)

3.692864271457086

In [ ]:
# #저장하기 
# df.to_csv(PATH+"_check(3.78).csv",index=False)  #체점 결과 
# submission_df=df[["Id","Predicted"]]
# submission_df.to_csv(PATH+"_submission.csv",index=False) #제출용